In [ ]:
# edit the FK tables to add the MC uncertainty. Follow https://pineappl.readthedocs.io/en/latest/introduction.html#How-can-I-edit-a-grid?

In [6]:
import pandas as pd
import pineappl
from pineappl.convolutions import Conv, ConvType
import lhapdf
import numpy as np

# Choose the PDF set to be used
pdfset = "NNPDF40_nnlo_as_01180"
# Use the central PDF, ie. member ID=0
pdf = lhapdf.mkPDF(pdfset, 0)


grid = pineappl.grid.Grid.read("/Users/jaco/Documents/physics_projects/theories_slim/data/grids/40009000/ATLAS_TTBAR_7TEV_TOT_X-SEC.pineappl.lz4")

conv_type = ConvType(polarized=False, time_like=False)
conv_object = Conv(convolution_types=conv_type, pid=2212)


predictions = grid.convolve(
    pdg_convs=[conv_object, conv_object],  # Similar convolutions for symmetric protons
    xfxs=[pdf.xfxQ2, pdf.xfxQ2],  # Similar PDF sets for symmetric protons
    alphas=pdf.alphasQ2,
)
df_preds = pd.DataFrame(
    {
        "bins": range(predictions.size),
        "predictions": predictions,
    }
)
df_preds

# df_bins = pd.DataFrame({"bin normalization": grid.bin_normalizations()})
# df_bins

LHAPDF 6.5.5 loading /Users/jaco/miniconda3/envs/nnpdf_dev/share/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331100


,bins,predictions
0,0,166.396535


In [16]:
# Import the Module to set bin specifications
from pineappl.boc import BinsWithFillLimits

# To get the bin configurations
bin_dims = grid.bin_dimensions()

bin_specs = np.array(grid.bin_limits())

# Extract the left & right bin limits
bin_limits = [
    [(bin_specs[b, d, 0], bin_specs[b, d, 1]) for d in range(bin_dims)]
    for b in range(grid.len())
]

# Set the normalization factor to `1`
#divide by 2
# TODO: find rescaling factors from MC
normalizations = [2.0 for _ in grid.bin_normalizations()]

# Instantiate the bin spec object
bin_configs = BinsWithFillLimits.from_limits_and_normalizations(
    limits=bin_limits,
    normalizations=normalizations,
)

# Set the updated bin configurations
grid.set_bwfl(bin_configs)

In [17]:
predictions2 = grid.convolve(
    pdg_convs=[conv_object, conv_object],  # Similar convolutions for symmetric protons
    xfxs=[pdf.xfxQ2, pdf.xfxQ2],  # Similar PDF sets for symmetric protons
    alphas=pdf.alphasQ2,
)
df_preds2 = pd.DataFrame(
    {
        "bins": range(predictions2.size),
        "predictions": predictions2,
    }
)
df_preds2

,bins,predictions
0,0,83.198268


In [ ]:
# write new grid
#grid.write_lz4("./LHCB_DY_8TEV_custom_normalizations.pineappl.lz4")